### Install the libraries

* **torch -** The core PyTorch deep learning framework used for building and training neural networks, featuring GPU-accelerated Tensor computation.

* **transformers -**	Hugging Face's library providing easy access to and use of thousands of pre-trained Transformer models (like LLMs, BERT, GPT) for various AI tasks.

* **datasets -**	Hugging Face's library for loading, sharing, and efficiently processing a vast collection of machine learning datasets.
* **trl	-** The Transformer Reinforcement Learning library used for aligning LLMs with human preferences via methods like Supervised Fine-Tuning (SFT) and Direct Preference Optimization (DPO).
* **unsloth -**	A highly optimized library that uses custom kernels to make LLM fine-tuning significantly faster and more memory-efficient (QLoRA) on consumer GPUs.
* **unsloth_zoo	-** A supplementary package for Unsloth that provides optimized model implementations and utilities to ensure maximum performance and compatibility with LLM architectures.

In [ ]:
! pip install torch transformers datasets trl unsloth unsloth_zoo

In [3]:
import torch
from unsloth import FastModel  # Unsloth fast loader + training utils
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
from datasets import load_dataset  # Hugging Face datasets
from trl import SFTTrainer  # Supervised fine-tuning trainer
from transformers import TrainingArguments  # Training hyperparameters

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# Minimal config (GPU expected). Adjust sizes: 270m, 1b, 4b, 12b, 27b
MODEL_NAME = "unsloth/gemma-3-270m-it"
MAX_SEQ_LEN = 2048
LOAD_IN_4BIT = True  # 4-bit quantized loading for low VRAM
LOAD_IN_8BIT = False  # 8-bit quantized loading for low VRAM
FULL_FINETUNING = False  # LoRA adapters (efficient) instead of full FT

In [5]:

def load_model_and_tokenizer():
    # Load Gemma 3 + tokenizer with desired context/quantization
    model, tokenizer = FastModel.from_pretrained(
        model_name=MODEL_NAME,
        max_seq_length=MAX_SEQ_LEN,
        load_in_4bit=LOAD_IN_4BIT,
        load_in_8bit=LOAD_IN_8BIT,
        full_finetuning=FULL_FINETUNING,
    )

    if not FULL_FINETUNING:
        # Add LoRA adapters on attention/MLP projections (PEFT)
        model = FastModel.get_peft_model(
            model,
            r=16,
            target_modules=[
                "q_proj", "k_proj", "v_proj", "o_proj",
                "gate_proj", "up_proj", "down_proj",
            ],
        )

    # Apply Gemma 3 chat template for correct conversation formatting
    tokenizer = get_chat_template(tokenizer, chat_template="gemma-3")
    return model, tokenizer

In [6]:
def prepare_dataset(tokenizer):
    # Load ShareGPT-style conversations and standardize schema
    dataset = load_dataset("mlabonne/FineTome-100k", split="train")
    dataset = standardize_sharegpt(dataset)
    # Render each conversation into a single training string
    dataset = dataset.map(
        lambda ex: {"text": [tokenizer.apply_chat_template(c, tokenize=False) for c in ex["conversations"]]},
        batched=True,
    )
    return dataset

In [7]:
def train(model, dataset):
    # Choose precision based on CUDA capabilities
    use_bf16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    use_fp16 = torch.cuda.is_available() and not use_bf16

    # Initialize the Supervised Fine-Tuning Trainer (SFTTrainer) from the TRL library
    trainer = SFTTrainer(
        model=model,                        # The pre-loaded language model. Often a 4-bit or 8-bit quantized model (e.g., from Unsloth)
        train_dataset=dataset,              # The processed dataset containing the examples for fine-tuning

        dataset_text_field="text",          # The name of the column in the dataset that holds the formatted text (instruction/response)
        max_seq_length=MAX_SEQ_LEN,         # The maximum number of tokens for the input sequence. Longer sequences are truncated.

        args=TrainingArguments(             # Define all training configuration arguments
            per_device_train_batch_size=2,  # The number of samples processed per GPU before calculating the gradient. Set low (e.g., 2) for large models to save VRAM.
            gradient_accumulation_steps=4,  # The number of batches to process before performing a weight update. Virtual batch size = 2 * 4 = 8. Used to simulate larger batches with less VRAM.
            warmup_steps=5,                 # The number of steps for the learning rate to linearly increase from 0.
            max_steps=60,                   # The total number of training steps to perform. Set to 60 for a quick test or small task.
            learning_rate=2e-4,             # The rate at which model weights are updated. 2e-4 is a common, optimized rate for QLoRA.
            bf16=use_bf16,                  # Enable bfloat16 mixed-precision training (preferred on modern GPUs for stability).
            fp16=use_fp16,                  # Enable float16 mixed-precision training (used when bf16 isn't supported).
            logging_steps=1,                # Log training metrics (like loss) after every single step.
            output_dir="outputs",           # The directory where model checkpoints and training logs will be saved.
        ),
    )
    trainer.train()

In [8]:
def main():
    # 1) Load model/tokenizer, 2) Prep data, 3) Train, 4) Save weights
    model, tokenizer = load_model_and_tokenizer()
    dataset = prepare_dataset(tokenizer)
    train(model, dataset)
    model.save_pretrained("finetuned_model")


if __name__ == "__main__":
    main()

==((====))==  Unsloth 2025.10.3: Fast Gemma3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/393M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sachin-tripathi (sachin-tripathi-aim) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.396000
2,2.850300
3,2.334100
4,2.158200
5,2.064900
6,2.380600
7,1.641600
8,2.460600
9,2.200400
10,2.304800
